In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from importlib import reload
%matplotlib inline
import emcee
import numpy as np
import matplotlib.pyplot as plt
from spectra import Spectrum

In [2]:
from numpy import exp, sqrt, pi
def lnL(theta, x, y, yerr):
    #a,b=theta
    amp,sigma,lambda_rest = theta
    #model = b * x + a
    model = (amp / (sqrt(2*pi) * sigma)) * exp(-(x-lambda_rest)**2 / (2*sigma**2))
    inv_sigma2 = 1.0/(yerr**2)
    return -0.5*(np.sum((y-model)**2*inv_sigma2))

def lnprior(theta):
    amp,sigma,lambda_rest = theta
    #amp,deltaLambda,sigma = theta
    if 0 < sigma < 10 and 4000 < lambda_rest < 7000:
        return 0.0
    return -np.inf

def lnprob(theta, x, y, yerr):
    """
    The likelihood to include in the MCMC.
    """
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnL(theta, x, y, yerr)

In [3]:
# problem3
from __future__ import print_function

import numpy as np
import emcee
import sys
import matplotlib.pyplot as plt
import corner


def run_emcee(x, y, y_obs, sigma, ml_result): #ml_result is the result object from lmfit

    # Set up the properties of the problem.
    ndim, nwalkers = 3, 200

    # Setup a bunch of starting positions.
    pos = [ml_result + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
        
    # Create the sampler.
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y_obs, sigma))

    sampler.run_mcmc(pos, 500)

    samples = sampler.chain[:, 50:, :].reshape((-1, 3))

    return sampler, samples



def show_walkers(chain, labels, savefile=None):

    nwalkers, nreps, ndim = chain.shape

    xval = np.arange(0, nreps)

    fig, ax = plt.subplots(ncols=1, nrows=ndim, figsize=(14, 4))
    for i_dim in range(ndim):
        ax[i_dim].set_ylabel(labels[i_dim])
        
        for i in range(100):
            ax[i_dim].plot(xval, chain[i, :, i_dim], color='black', alpha=0.5)

    if savefile is not None:
        plt.savefig(savefile)

    return fig, ax


def show_corner_plot(samples,labels,truths, savefile=None):

    fig = corner.corner(samples, labels=labels, truths=truths, quantiles=[0.16,0.5,0.84])

    
    if savefile is not None:
        plt.savefig(savefile)

    return fig

In [6]:
spec = Spectrum('src/data-spectrum/new/idid000007077jd2458163p6789f000.fits')
#spec.plot('-r',4340,4780)
#lambda0 = 6562.85175
lambda0 = 4861.297
#spec.gfitting([lT],False)
#spec.plot('-r',4840,4880)


In [7]:

#print(dlambda)
#print(sigma)
data = spec.fit_unique(lambda0,4840,4880)
amp = data['result'].params['amp'].value
sigma = data['result'].params['sigma'].value
lambda_rest = data['result'].params['lambda0'].value
#print(amp,sigma,lambda_rest)
#data = np.random.normal(loc=100.0, scale=3.0, size=1000)

In [ ]:
# See ml_illustration for the run_ml() function that calculates these numbers.
p_initial = [amp,sigma,lambda_rest]#values from result object

sampler_gaussian, samples_gaussian = run_emcee(data['x'], data['y'], data['y'], data['sigma'], p_initial)

### Show the chains

In [ ]:
fig, axes = show_walkers(sampler_gaussian.chain, ['amp', 'sigma', 'lambda_rest'])


### Show a corner plot

In [ ]:
fig = show_corner_plot(samples_gaussian, ['amp','sigma','lambda_rest'], p_initial)

In [120]:
'''
MCMC fitting template. 
This template fits a 1-d gaussian, if you 
figure out how to use it for more complicated distributions
I'd appreciate if you let me know :)
banados@mpia.de
'''

#First let's create a gaussian data

data = np.random.normal(loc=100.0, scale=3.0, size=1000)

# Then, define the probability distribution that you would like to sample.
def lnprob(p, vec):
    diff = vec-p[0]
    N = len(vec)
    return -0.5 * N * np.log(2 * np.pi) - N * np.log(p[1]) - 0.5 \
                                    * np.sum(( (vec - p[0]) / p[1] ) ** 2)
    
               
# We'll sample a Gaussian which has 2 parameters: mean and sigma...
ndim = 2

# We'll sample with 250 walkers. (nwalkers must be an even number)
nwalkers = 250

# Choose an initial set of positions for the walkers.
p0 = [np.random.rand(ndim) for i in range(nwalkers)]

# Initialize the sampler with the chosen specs.
#The "a" parameter controls the step size, the default is a=2,
#but in this case works better with a=4 see below or page 10 in the paper
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=[data], a=4)

# Run 200 steps as a burn-in.
print("Burning in ...")
pos, prob, state = sampler.run_mcmc(p0, 200)

# Reset the chain to remove the burn-in samples.
sampler.reset()

# Starting from the final position in the burn-in chain, sample for 1000
# steps. (rstate0 is the state of the internal random number generator)
print("Running MCMC ...")
pos, prob, state = sampler.run_mcmc(pos, 1000, rstate0=state)

# Print out the mean acceptance fraction. In general, acceptance_fraction
# has an entry for each walker so, in this case, it is a 250-dimensional
# vector.
af = sampler.acceptance_fraction
print("Mean acceptance fraction:", np.mean(af))
af_msg = '''As a rule of thumb, the acceptance fraction (af) should be 
                            between 0.2 and 0.5
            If af < 0.2 decrease the a parameter
            If af > 0.5 increase the a parameter
            '''

print(af_msg)

# If you have installed acor (http://github.com/dfm/acor), you can estimate
# the autocorrelation time for the chain. The autocorrelation time is also
# a vector with 10 entries (one for each dimension of parameter space).
try:
    print("Autocorrelation time:", sampler.acor)
except ImportError:
    print("You can install acor: http://github.com/dfm/acor")
    
maxprob_indice = np.argmax(prob)

mean_fit, sigma_fit = pos[maxprob_indice]

print("Estimated parameters: mean, sigma = %f, %f" % (mean_fit, sigma_fit))

mean_samples = sampler.flatchain[:,0]
sigma_samples = sampler.flatchain[:,1]

mean_std = mean_samples.std()
sigma_std = sigma_samples.std()

print("parameters' error: mean, sigma = %f, %f" % (mean_std, sigma_std))

# Finally, you can plot the projected histograms of the samples using
# matplotlib as follows (as long as you have it installed).
try:
    import matplotlib.pyplot as plt
except ImportError:
    print("Try installing matplotlib to generate some sweet plots...")
else:
    plt.hist(mean_samples, 100)
    plt.title("Samples for mean")
    plt.show()
    plt.title("Samples for sigma")
    plt.hist(sigma_samples, 100)
    plt.show()

Burning in ...


ValueError: Probability function returned NaN